In [1]:
import scipy.io as sio
from pathlib import Path
import numpy as np


mask = np.array([np.ones(32), np.ones(32), np.ones(32),np.concatenate((np.zeros(14), np.ones(18))),
                 np.concatenate((np.zeros(14), np.ones(18))),
                 np.concatenate((np.zeros(14), np.ones(18))),
                 np.ones(32), np.ones(32), np.ones(32),
                 np.concatenate((np.zeros(14), np.ones(18))),
                 np.ones(32), np.ones(32), np.ones(32),
                 np.concatenate((np.zeros(14), np.ones(18))),
                 np.concatenate((np.zeros(14), np.ones(18))),
                 np.ones(32), np.ones(32), np.ones(32),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3))),
                 np.concatenate((np.zeros(25), np.ones(4), np.zeros(3)))]).astype(np.bool)
mask = mask.reshape((1024,))

C:\Users\BCI-AD~1\AppData\Local\Temp/ipykernel_22020/3725482898.py:28: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.concatenate((np.zeros(25), np.ones(4), np.zeros(3)))]).astype(np.bool)


In [2]:
def normalize(pressure):
    """
    Scales each array of the given array of arrays to the range [0, 1]
    Only considers values in the same tactile frame
    """
    normalized_p = np.copy(pressure)
    for i, press in enumerate(pressure):
        min_p = np.min(press)
        normalized_p[i] = (press - min_p) / np.max(press - min_p)
    
    return normalized_p
 

def boost(pressure): 
    """
    The higher a value is from the mean of the frame, the more it gets boosted.
    The idea is that tactile features are robuster
    """
    for press in pressure:
        mean_p = np.mean(press[mask])
        boost_mask = press > mean_p
        press[boost_mask] = list(map(lambda x: 4*(x-mean_p), press[boost_mask]))
        
    return pressure

In [4]:
filename = Path('C:/Users/bci-admin/Downloads/smarthand_dataset.mat')

data = sio.loadmat(filename, squeeze_me=True)
pressure = data['tactile_data']
# Scale data to the range [0, 1]
pressure = np.clip((pressure.astype(np.float32)-1500)/(2700-1500), 0.0, 1.0)
#pressure = normalize(pressure.astype(np.float32))
#pressure = np.exp2(pressure)
#pressure = np.clip((pressure-1), 0.0, 1.0)
pressure = boost(pressure)
pressure = np.clip(pressure, 0.0, 1.0)
object_id = data['object_id']

pressure[:, ~mask] = 0.0

In [ ]:
print(pressure.shape)

### 5 Frames as a group, N_frames = 5

In [150]:
N=5
sample = []
for i in range(int(pressure.shape[0]/N)):
    sample.append(pressure[N*i:N*i+N,:])
    
sample = np.array(sample)
print(sample.shape)

(67997, 5, 1024)


In [151]:
sample = sample.reshape((-1, 32*32*N))
print(sample.shape)

(67997, 5120)


In [157]:
object_sample = object_id[0:-5:5]
print(object_sample.shape)

(67997,)


In [158]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sample), np.array(object_sample), test_size=0.2, random_state=42)

In [160]:
X_train.shape

(54397, 5120)

In [161]:
y_train.shape

(54397,)

In [162]:
X_test.shape

(13600, 5120)

In [163]:
y_test.shape

(13600,)

In [164]:
# N=5
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='linear') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8755147058823529


### 2 Frames as a group, N_frames = 2

In [165]:
N=2
sample = []
for i in range(int(pressure.shape[0]/N)):
    sample.append(pressure[N*i:N*i+N,:])
    
sample = np.array(sample)
print(sample.shape)
sample = sample.reshape((-1, 32*32*N))
print(sample.shape)
object_sample = object_id[0:-N:N]
print(object_sample.shape)

(169994, 2, 1024)
(169994, 2048)
(169994,)


In [166]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sample), np.array(object_sample), test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(135995, 2048)
(135995,)


In [167]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='linear') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8794964557781111


### 3 Frames as a group, N_frames = 3

In [168]:
N=3
sample = []
for i in range(int(pressure.shape[0]/N)):
    sample.append(pressure[N*i:N*i+N,:])
    
sample = np.array(sample)
print(sample.shape)
sample = sample.reshape((-1, 32*32*N))
print(sample.shape)
object_sample = object_id[0:-N:N]
print(object_sample.shape)

(113329, 3, 1024)
(113329, 3072)
(113329,)


In [169]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sample), np.array(object_sample), test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(90663, 3072)
(90663,)


In [170]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='linear') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8814523956586958


### 4 Frames as a group, N_frames = 4

In [14]:
N=4
sample = []
for i in range(int(pressure.shape[0]/N)):
    sample.append(pressure[N*i:N*i+N,:])
    
sample = np.array(sample)
print(sample.shape)
sample = sample.reshape((-1, 32*32*N))
print(sample.shape)
object_sample = object_id[0:-N:N]
print(object_sample.shape)

(84997, 4, 1024)
(84997, 4096)
(84997,)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sample), np.array(object_sample), test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(67997, 4096)
(67997,)


In [173]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='linear') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8831764705882353


In [16]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='rbf') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9334705882352942


In [174]:
N=5
sample = []
for i in range(int(pressure.shape[0]/N)):
    sample.append(pressure[N*i:N*i+N,:])
    
sample = np.array(sample)
print(sample.shape)
sample = sample.reshape((-1, 32*32*N))
print(sample.shape)
object_sample = object_id[0:-N:N]
print(object_sample.shape)

(67997, 5, 1024)
(67997, 5120)
(67997,)


In [175]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sample), np.array(object_sample), test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(54397, 5120)
(54397,)


In [176]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='linear') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8755147058823529


In [5]:
N=6
sample = []
for i in range(int(pressure.shape[0]/N)):
    sample.append(pressure[N*i:N*i+N,:])
    
sample = np.array(sample)
print(sample.shape)
sample = sample.reshape((-1, 32*32*N))
print(sample.shape)
object_sample = object_id[0:-N:N]
print(object_sample.shape)

(56664, 6, 1024)
(56664, 6144)
(56664,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(sample), np.array(object_sample), test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(45331, 6144)
(45331,)


In [8]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='linear') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8807906114885732


In [6]:
N=50
sample = []
for i in range(int(pressure.shape[0]/N)):
    sample.append(pressure[N*i:N*i+N,:])
    
sample = np.array(sample)
print(sample.shape)
sample = sample.reshape((-1, 32*32*N))
print(sample.shape)
object_sample = object_id[0:-N:N]
print(object_sample.shape)

(6799, 50, 1024)
(6799, 51200)
(6799,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(sample), np.array(object_sample), test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(5439, 51200)
(5439,)


In [8]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='linear') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7154411764705882


In [9]:
N=100
sample = []
for i in range(int(pressure.shape[0]/N)):
    sample.append(pressure[N*i:N*i+N,:])
    
sample = np.array(sample)
print(sample.shape)
sample = sample.reshape((-1, 32*32*N))
print(sample.shape)
object_sample = object_id[0:-N:N]
print(object_sample.shape)

(3399, 100, 1024)
(3399, 102400)
(3399,)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(sample), np.array(object_sample), test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(2719, 102400)
(2719,)


In [11]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='linear') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7029411764705882


In [12]:
N=10
sample = []
for i in range(int(pressure.shape[0]/N)):
    sample.append(pressure[N*i:N*i+N,:])
    
sample = np.array(sample)
print(sample.shape)
sample = sample.reshape((-1, 32*32*N))
print(sample.shape)
object_sample = object_id[0:-N:N]
print(object_sample.shape)

(33998, 10, 1024)
(33998, 10240)
(33998,)


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(sample), np.array(object_sample), test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(27198, 10240)
(27198,)


In [14]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='linear') 

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.856764705882353
